In [21]:
import csv
import collections
import random as rn
from numpy import concatenate, random
from numpy.random import randint
import copy

In [22]:
# global variables
classroom_List = [("P411", 0), ("P412", 1), ("P413", 2), ("P414", 3), ("P415", 4),
                  ("P416", 5), ("P417", 6), ("P418", 7), ("P419", 8), ("P420", 9)]  # 10 classrooms
totalClassRooms = len(classroom_List)
days_List = [("Monday", 0), ("Tuesday", 1), ("Wednesday", 2),
             ("Thursday", 3), ("Friday", 4)]  # 5 days a week
totalDays = len(days_List)
examStartTimings = [(10, 0), (2, 1)]  # 10:00 AM, 2:00 PM
totalExamStartTimings = len(examStartTimings)
units = []  # list of units
capacity = 30
instructors = []  # list of instructors
instructors_Count = -1  # number of instructors
student_Units_List = []  # list of student_Unitss
Chromosome = collections.namedtuple(
    'Population', 'chromosome value')  # chromosome

# genetic algorithm parameters
population_size = 0
crossover_probability, mutation_probability = 0.0, 0.0


In [23]:
# File reader function
def load_Data():
    # units from file
    with open('units.csv') as file:
        reader = csv.reader(file)
        count = 0
        for row in reader:
            if len(row) != 0:
                temp_course = unit(row[0], row[1], count)
                if temp_course not in units:
                    units.append(temp_course)
                    count += 1

    #  instructors from file
    with open('tutor.csv') as file:
        reader = csv.reader(file)
        count = 0
        for row in reader:
            if len(row) != 0:
                check = False
                for instructor in instructors:
                    if instructor[0] == row[0]:
                        check = True
                        break
                if not check:
                    instructors.append((row[0], count))
                    count += 1
    global instructors_Count
    instructors_Count = len(instructors)

    # students from file
    with open('student_names.csv') as file:
        reader = csv.reader(file)
        for row in reader:
            if len(row) != 0:
                temp_student_Units = student_Units(row[0], [])
                if temp_student_Units not in student_Units_List:
                    student_Units_List.append(temp_student_Units)

    # student_units from file
    with open('student_units.csv') as file:
        reader = csv.reader(file)
        for row in reader:
            if len(row) != 0 and row[0] != '':
                for i in student_Units_List:
                    if i.studentName == row[1] and row[2] not in i.student_in_Unit:
                        i.student_in_Unit.append(row[2])


In [24]:
# Store unit class
class unit:
    # Constructor for unit
    def __init__(self, code, name, number):
        self.courseCode = code
        self.courseName = name
        self.number = number

    # Print unit
    def __repr__(self):
        return '({0},{1},{2})'.format(self.courseCode, self.courseName, self.number)

    # Compare unit
    def __eq__(self, other):
        return self.courseName == other.courseName and self.courseCode == other.courseCode


In [25]:
# store student_Units class
class student_Units:
    #  Constructor for student_Units
    def __init__(self, Name, courseCodes):
        self.studentName = Name
        self.student_in_Unit = courseCodes.copy()

    # Print student_Units
    def __repr__(self):
        return '({0},{1})'.format(self.studentName, self.student_in_Unit)

    # Compare student_Units
    def __eq__(self, other):
        if self.studentName == other.studentName and len(self.student_in_Unit) == len(other.student_in_Unit):
            count = 0
            for i in range(len(self.student_in_Unit)):
                if self.student_in_Unit[i] == other.student_in_Unit[i]:
                    count += 1
            if count == len(self.student_in_Unit):
                return True
        return False


In [26]:
# store Exam class
class Exam:
    # define exam constructor
    # startTime, roomNo, day, invigilator
    def __init__(self, startTime, roomNo, day, invigilator):
        self.startTime = startTime      # tuple of start time
        self.roomNo = roomNo.copy()  # list of rooms
        self.day = day
        self.invigilator = invigilator.copy()
        self.binary = []

    # exam print
    def __repr__(self):
        return '(\n{0}, {1}, {2}, {3}), \n{4}'.format(self.startTime,
                                                      self.roomNo, self.day, self.invigilator, self.binary)

    # exam compare
    def __eq__(self, other):
        if len(self.roomNo) == len(other.roomNo) and self.startTime == other.startTime and self.day == other.day \
                and len(self.invigilator) == len(other.invigilator):  # and self.roomNo == other.roomNo and self.invigilator == other.invigilator:
            count = 0
            count1 = 0
            for i in range(len(self.roomNo)):
                if self.roomNo[i] == other.roomNo[i]:
                    count += 1
            for i in range(len(self.invigilator)):
                if self.invigilator[i] == other.invigilator[i]:
                    count1 += 1
            if count == len(self.roomNo) and count1 == len(self.invigilator):
                return True
        return False


In [27]:
# Generating random exam
def randomExam(index):
    courseCode = units[index].courseCode
    students = [
        x for x in student_Units_List if courseCode in x.student_in_Unit]
    total_students = len(students)

    # assigning rooms
    roomNo = []
    while total_students > 0:
        temp = classroom_List[rn.randrange(0, totalClassRooms)]
        while temp in roomNo:
            temp = classroom_List[rn.randrange(0, totalClassRooms)]
        roomNo.append(temp)
        total_students -= capacity

    # set time for exam (10:00 - 12:00, 2:00 - 4:00)
    startTime = examStartTimings[rn.randrange(0, totalExamStartTimings)]
    if startTime[0] > 12:
        startTime -= 12

    # set day from monday to friday
    day = days_List[rn.randrange(0, totalDays)]

    # assign invigilator to exam room
    invigilator = []
    for i in range(len(roomNo)):
        temp = instructors[rn.randrange(0, instructors_Count)]
        while temp in invigilator:
            temp = instructors[rn.randrange(0, instructors_Count)]
        invigilator.append(temp)
    return Exam(startTime, roomNo, day, invigilator)


In [28]:
# population generator function
def gen_Pop(size):
    new_pop = []

    #   generate random exam for each unit
    for i in range(size):
        timeTable = []
        for j in range(len(units)):
            timeTable.append(randomExam(j))
    #  append new chromosome to population
        new_pop.append(
            Chromosome(
                chromosome=timeTable,
                value=-1
            )
        )
    return new_pop


In [29]:
# mutate chromosomes
def mutate_Pop(chromosome):
    if random.randint(0, 100) <= mutation_probability * 100:  # chance of mutation
        gene = random.randint(0, len(units) - 1)  # random gene
        chromosome[gene] = randomExam(gene)  # mutate gene
    return chromosome


In [30]:
# Apply Crossover on population
def crossover_Pop(population):
    crossover_population = []

    # while crossover population is less than population
    while len(crossover_population) < len(population):
        if randint(0, 100) <= crossover_probability * 100:
            # select mama and papa
            parent_a = randint(0, len(population) - 1)
            parent_b = randint(0, len(population) - 1)

            # get some genes from mama and some from papa
            chromosome_a = copy.deepcopy(concatenate((population[parent_a].chromosome[:int(len(units) / 2)],
                                                      population[parent_b].chromosome[int(len(units) / 2):])))
            chromosome_a = mutate_Pop(chromosome_a) #mutant child 1 

            chromosome_b = copy.deepcopy(concatenate((population[parent_b].chromosome[:int(len(units) / 2)],
                                                      population[parent_a].chromosome[int(len(units) / 2):])))
            chromosome_b = mutate_Pop(chromosome_b) # mutant child 2

            crossover_population.append(Chromosome(
                chromosome=chromosome_a,
                value=-1
            ))
            crossover_population.append(Chromosome(
                chromosome=chromosome_b,
                value=-1
            ))

    # calculate fitness of crossover population
    crossover_population = calculate_fitness(crossover_population)
    # sort crossover population
    population = population + crossover_population
    return population


In [31]:

# roulette wheel selection
def roulette_wheel_selection(population):
    # calculate fitness of population
    population_fitness = sum([individual.value for individual in population])
    # calculate probability of each chromosome
    chromosome_probabilities = [
        round(individual.value / population_fitness, 5) for individual in population]

    copy_probabilities = chromosome_probabilities.copy()  # copy probabilities
    copy_probabilities.sort()  # sort probabilities
    for i in range(len(copy_probabilities)):  # calculate cumulative probability
        if i != 0:
            copy_probabilities[i] = round(
                copy_probabilities[i] + copy_probabilities[i - 1], 5)

    # select chromosomes based on probability
    selected_population = []
    for i in range(population_size):
        index = -1
        random_probability = round(random.uniform(0, 1), 5)
        for j in range(len(copy_probabilities)):
            if random_probability <= copy_probabilities[j]:
                value = copy_probabilities[j]
                if j != 0:
                    value = round(value - copy_probabilities[j - 1], 5)
                index = chromosome_probabilities.index(value)
                break
        selected_population.append(population[index])
    return selected_population


In [32]:
# check student constraint
def one_exam_student_check(chromosome):
    violation_count = 0
    for i in range(0, len(student_Units_List)):
        days_arr = []
        for j in range(0, len(chromosome)):
            if units[j].courseCode in student_Units_List[i].student_in_Unit:
                if (chromosome[j].day[1], chromosome[j].startTime) not in days_arr:
                    days_arr.append(
                        (chromosome[j].day[1], chromosome[j].startTime))
                else:
                    violation_count += 1
    return violation_count


In [33]:
# find fittest individual in population
def find_fittest_individual(population):
    highest_value = 0
    highest_index = 0
    for i in range(len(population)):
        if population[i].value > highest_value:
            highest_value = population[i].value
            highest_index = i
    return population[highest_index]


In [34]:
# check room constraint
def checkRooms(chromosome):
    violation_count = 0
    for gene in chromosome:
        for room in gene.roomNo:
            for gene1 in chromosome:
                for room1 in gene1.roomNo:
                    if gene != gene1:
                        if room == room1 and gene.day == gene1.day and gene.startTime == gene1.startTime:
                            violation_count += 1
    return violation_count // 2


In [35]:
# check invigilation constraint
def checkInvigilation(chromosome):
    violation_count = 0
    data = [(individual.invigilator, individual.day, individual.startTime)
            for individual in chromosome]
    for i in range(len(data)):
        for invigilator in data[i][0]:
            for j in range(len(data)):
                if i != j and invigilator in data[j][0] and data[i][1] == data[j][1] and data[i][2] == data[j][2]:
                    violation_count += 1
    return violation_count // 2


In [36]:
# Fitness function to calculate value of chromosome
def calculate_value(chromosome):
    value = 512  # max value
    value -= checkInvigilation(chromosome)
    value -= checkRooms(chromosome)
    value -= one_exam_student_check(chromosome)

    # encode to binary
    for i in range(len(chromosome)):
        chromosome[i].binary.clear()
        chromosome[i].binary.append(bin(units[i].number)[2:].zfill(6))
        chromosome[i].binary.append(
            bin(chromosome[i].startTime[1])[2:].zfill(6))
        tempRoom = []
        for room in chromosome[i].roomNo:
            tempRoom.append(bin(room[1])[2:].zfill(6))
        chromosome[i].binary.append(tempRoom)
        chromosome[i].binary.append(bin(chromosome[i].day[1])[2:].zfill(6))
        tempInvigilator = []
        for invigilator in chromosome[i].invigilator:
            tempInvigilator.append(bin(invigilator[1])[2:].zfill(6))
        chromosome[i].binary.append(tempInvigilator)
    return value


In [37]:
# check if chromosome is valid ( no constraint violation )
def calculate_fitness(population):
    for i in range(len(population)):
        v = calculate_value(population[i].chromosome)
        population[i] = Chromosome(
            chromosome=population[i].chromosome,
            value=v
        )
    return population


In [38]:
# printing the final schedule
def display_schedule(best_solution):
    count = 0
    temp_day = -1

    week = 1
    last_week_entry = 0
    weekflag = False

    max_invigilator_length = 0
    for i in instructors:
        if len(i[0]) > max_invigilator_length:
            max_invigilator_length = len(i[0])

    max_length = 0
    for i in units:
        if len(i.courseName) > max_length:
            max_length = len(i.courseName)

    # deep copy to avoid changing the original chromosome
    best_solution_copy = copy.deepcopy(best_solution)
    best_solution_copy = Chromosome(sorted(best_solution_copy.chromosome, key=lambda x: (
        x.day[1], x.startTime[1])), best_solution_copy.value)  # sort by day and time

    # formatting the schedule
    print("\n\nSCHEDULE:\n--------")
    for i in best_solution_copy.chromosome:
        curr_day = i.day[1]
        if curr_day >= last_week_entry:
            if count != 0:
                print(end="\t\t")
                for j in range(0, 41 + max_length + max_invigilator_length):
                    print(end="-")
                print()

            print("\nWeek", week)
            print("------")
            week += 1
            last_week_entry += 5
            weekflag = True

        if temp_day != curr_day:
            if count != 0 and weekflag == False:
                print(end="\t\t")
                for j in range(0, 41 + max_length + max_invigilator_length):
                    print(end="-")
                print()

            print(end="\n\t\t")
            print(i.day[0])
            weekflag = False

        print(end="\t\t")
        for j in range(0, 41 + max_length + max_invigilator_length):
            print(end="-")
        print()

        print(end="\t\t")
        for j in range(0, len(units)):
            if best_solution.chromosome[j] == i:
                ind = j
                break
        print("|", units[ind].courseCode, "|",
              units[ind].courseName, end="")

        if len(units[ind].courseCode) == 5:
            for j in range(0, max_length-len(units[ind].courseName)):
                print(end=" ")
        else:
            for j in range(0, max_length-len(units[ind].courseName)-1):
                print(end=" ")
        print(end=" | ")

        if i.startTime[1] == 0:
            print(end="10:00 AM | ")
        else:
            print(end="2:00 PM | ")

        for j in range(0, len(i.roomNo)):
            if j != 0:
                print(end="\t\t")
                for k in range(0, 23+max_length):
                    print(end=" ")

            print("Room", end="")
            print(":", i.roomNo[j][0], "|", i.invigilator[j][0], end="")
            for k in range(0, max_invigilator_length - len(i.invigilator[j][0]) + 1):
                print(end=" ")
            print("|")

        temp_day = curr_day
        count += 1

    print(end="\t\t")
    for j in range(0, 41 + max_length + max_invigilator_length):
        print(end="-")
    print()


In [39]:
# run the genetic algorithm
def genetic_Algorithm():
    # initialising population and generation
    # population size set in main function
    population = gen_Pop(population_size)
    generation = 1
    best_solution = None

    # calculating fitness of initial population
    population = calculate_fitness(population)

    # loop until solution is found
    while True:
        # mutate and crossover
        population = crossover_Pop(population)
        population = roulette_wheel_selection(population)
        # find fittest individual
        candidate = find_fittest_individual(population)

        # save best solution so far
        if best_solution is None:
            best_solution = candidate
        elif candidate.value > best_solution.value:
            best_solution = candidate

        # print best solution so far and generation number
        if generation % 1 == 0 or generation == 1:
            print('\nCurrent generation: {}'.format(generation))
            print('Best solution so far: {}, Goal: 512'.format(best_solution.value))

        # VICTORY CONDITION #

                                                                                                         # Hard constraints #
                                                                            #An exam will be scheduled for each unit. Unit name and code are presented in units.csv file.
                                                                            #A student is enrolled at least one unit, but can be enrolled upto four units.
                                                                            #A student cannot appear in more than one exam at a time.
                                                                            #Exam won’t be held on the weekends i.e., on Saturday and Sunday.
                                                                            #Each exam must be invigilated by a tutor. You can get tutor information using tutor.csv file. You should display tutor
                                                                            #name in the output.
                                                                            #A tutor invigilates one exam at a time.
                                                                            #Each exam must be conducted between 10:00 am to 4:00 pm.
        if best_solution.value == 512:
            print('\nFound Solution:')
            print('Value: {}, Goal: 512'.format(best_solution.value))
            # hard constraints check
            print("\nHard Constraints:")
            print("1: Atleast 1 exam for each unit: ✔")
            if one_exam_student_check(best_solution.chromosome) == 0:
                print("2: A student can not give more than one exam at a time: ✔")
            else:
                print("2: A student can not give more than one exam at a time: ❌")
            print("3: All exams must be either 10-12 or 2-4: ✔")
            if checkInvigilation(best_solution.chromosome) == 0:
                print("4: A tutor cannot be in 2 places at once: ✔")
            else:
                print("4: A tutor cannot be in 2 places at once: ❌")
            if checkRooms(best_solution.chromosome) == 0:
                print("5: No exams are scheduled for the same room and same time: ✔")
            else:
                print("5: No exams are scheduled for the same room and same time: ❌")

                                                                                                            # Soft constraints #
                                                                    # Student should not sit in more than one exam consecutively in a day. In other word, if a student sits in exam on
                                                                    # Monday in the morning slot, then don’t schedule another exam for the same student in the afternoon slot.
                                                                    # Try to assign equal number of invigilation duties to each tutor.
            # soft constraints check
            print("\nSoft Constraints:")
            print("1: An exam will be scheduled for each unit: ✔")
            print("2: A student shouldn't sit more than 1 exam a day: ✔")
            print("\n TODO: units with no exam still show up, but they aren't assigned a room or invigilator (which is fine i guess?)")

            display_schedule(best_solution)
            break
        generation += 1  # increment generation


In [40]:
if __name__ == "__main__":
    #  Loading data from csv files
    load_Data()

    # Initializing variables for Genetic Algorithm
    population_size = random.randint(50,250)
    crossover_probability = 0.9
    mutation_probability = 0.1

    #  Printing parameters of Genetic Algorithm
    print('----- Generated Parameters -----')
    print('Population size......: {}'.format(population_size))
    print('Crossover probability: {}'.format(crossover_probability))
    print('Mutation probability.: {}'.format(mutation_probability))

    # Run it baby
    genetic_Algorithm()


----- Generated Parameters -----
Population size......: 195
Crossover probability: 0.9
Mutation probability.: 0.1

Current generation: 1
Best solution so far: 512, Goal: 512

Found Solution:
Value: 512, Goal: 512

Hard Constraints:
1: Atleast 1 exam for each unit: ✔
2: A student can not give more than one exam at a time: ✔
3: All exams must be either 10-12 or 2-4: ✔
4: A tutor cannot be in 2 places at once: ✔
5: No exams are scheduled for the same room and same time: ✔

Soft Constraints:
1: An exam will be scheduled for each unit: ✔
2: A student shouldn't sit more than 1 exam a day: ✔

 TODO: units with no exam still show up, but they aren't assigned a room or invigilator (which is fine i guess?)


SCHEDULE:
--------

Week 1
------

		Monday
		-------------------------------------------------------------------------------------------
		| CIS313 | Discrete Mathematics                       | 10:00 AM | Room: P416 | Tutor26 |
		------------------------------------------------------------